In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

In [2]:
# File paths
data_migraine_path = '/data/caysar9/data/long_migraine.csv'
data_sleep_path = '/data/caysar9/data/long_sleep.csv'
final_output_path = '/data/caysar9/results/final_minimum_migraine_sleep.csv'

In [3]:
# Load migraine data
print("Loading migraine data...")
data_migraine = pd.read_csv(data_migraine_path)

data_migraine.head()

Loading migraine data...


,hashed_uid,hashed_mid,age,gender,country,creationtime_local,starttime_local,endtime_local,pain_intensity,attack_type,symptoms,triggers,reliefs,pain_position,affected_activities,menstruation_status
0,dc4881d4872ae95fbc68736bf25d084676642e65c7d450...,8515e0cc14e74dc97d80db54b03d97208e01c04cca260c...,47.0,F,United States,2020-11-20 13:06:26.215,2020-11-20 12:30:00,2020-11-22 14:37:00,4.0,migraine,"nasal congestion,pressure,sensitivity to light",NaN,NaN,NaN,NaN,NaN
1,dc4881d4872ae95fbc68736bf25d084676642e65c7d450...,2cf353f70fe5b56c11fb7e8ef014bdb53669180acdc299...,49.0,F,United States,2022-09-06 09:17:34.845,2022-09-06 07:15:00,2022-09-08 07:54:00,6.0,"migraine,migraine with aura","frequent urination,nasal congestion,nausea,rin...","allergies,lack of sleep",no relief,"left_back_head,left_head,right_back_head,right...",NaN,menstrualcyclestatus:no
2,dc4881d4872ae95fbc68736bf25d084676642e65c7d450...,41493d5c9ba8a94161a4e14823d6424ac5044648cf5068...,48.0,F,United States,2021-09-19 22:35:39.053,2021-09-19 22:21:00,2021-09-22 13:27:00,6.0,migraine,neck pain,lack of sleep,stay indoor,"left_back_head,left_eye,left_head,right_back_h...",NaN,menstrualcyclestatus:no
3,dc4881d4872ae95fbc68736bf25d084676642e65c7d450...,89241d94a0af1f317f0c3248102a7c096dfaaaa1018a92...,49.0,F,United States,2022-08-09 14:52:59.914,2022-08-09 10:53:00,2022-08-13 08:39:00,6.0,migraine,"nausea,ringing in ears,sensitivity to light,se...",weather,hot bath,"left_back_head,left_eye,left_head,left_temple,...",NaN,menstrualcyclestatus:no
4,337ca43d6b0956488b93fe174a646c9a8f80288b30aefb...,7e5421d9f0866b0af6dce52c99655ac0ee0cec8f20474c...,34.0,F,United States,2021-06-05 23:53:27.319,2021-06-05 23:10:00,2021-06-06 05:10:00,6.0,migraine,"neck pain,runny nose",NaN,sleep,right_temple,not affected,menstrualcyclestatus:no


In [4]:
data_migraine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 697885 entries, 0 to 697884
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   hashed_uid           697885 non-null  object 
 1   hashed_mid           697885 non-null  object 
 2   age                  289327 non-null  float64
 3   gender               697885 non-null  object 
 4   country              697885 non-null  object 
 5   creationtime_local   697885 non-null  object 
 6   starttime_local      697885 non-null  object 
 7   endtime_local        697011 non-null  object 
 8   pain_intensity       683630 non-null  float64
 9   attack_type          651131 non-null  object 
 10  symptoms             624323 non-null  object 
 11  triggers             569461 non-null  object 
 12  reliefs              543133 non-null  object 
 13  pain_position        631018 non-null  object 
 14  affected_activities  523422 non-null  object 
 15  menstruation_stat

In [5]:
# Preprocess migraine dataset
print("Preprocessing migraine dataset...")
print("Dataset starting shape: ", data_migraine.shape)

Preprocessing migraine dataset...
Dataset starting shape:  (697885, 16)


In [6]:
# Selecting columns of interest
migraine_variables_of_interest = [
    "hashed_uid", "hashed_mid", "age", "gender", "country", "creationtime_local",
    "starttime_local", "endtime_local", "pain_intensity", "attack_type", "triggers", "reliefs", "affected_activities",
]
data_migraine = data_migraine[migraine_variables_of_interest]

# Convert to datetime and create year/month fields
data_migraine["creationtime_local"] = pd.to_datetime(data_migraine["creationtime_local"], format="ISO8601")
data_migraine["year"] = data_migraine["creationtime_local"].dt.year
data_migraine["month"] = data_migraine["creationtime_local"].dt.month
data_migraine["year_month"] = data_migraine["creationtime_local"].dt.to_period("M")

# Select only migraine attacks
data_migraine = data_migraine[data_migraine["attack_type"].str.contains("migraine", case=False, na=False)]

print("Dataset shape after filtering only migraine: ", data_migraine.shape)
print()

Dataset shape after filtering only migraine:  (467368, 16)



In [7]:
# Filter age range
data_migraine = data_migraine[(data_migraine["age"] <= 65) & (data_migraine["age"] >= 18)]

print("Dataset shape after filtering age: ", data_migraine.shape)
print()

# Drop duplicates based on creationtime_local
data_migraine.drop_duplicates(subset=["hashed_uid", "creationtime_local"], inplace=True)

# Drop duplicates in migraine id
data_migraine.drop_duplicates(subset=["hashed_uid", "hashed_mid"], inplace=True)

# Drop duplicates based on start and end days
data_migraine["starttime_local_to_day"] = pd.to_datetime(data_migraine["starttime_local"], format="ISO8601").dt.to_period("D")
data_migraine["endtime_local_to_day"] = pd.to_datetime(data_migraine["endtime_local"], format="ISO8601").dt.to_period("D")
data_migraine.drop_duplicates(subset=["hashed_uid", "starttime_local_to_day"], inplace=True)
data_migraine.drop_duplicates(subset=["hashed_uid", "endtime_local_to_day"], inplace=True)

# Calculate attack duration in days
data_migraine["attack_duration"] = pd.to_datetime(data_migraine["endtime_local"], format="ISO8601") - pd.to_datetime(data_migraine["starttime_local"], format="ISO8601")
data_migraine["attack_duration_days"] = data_migraine["attack_duration"].dt.total_seconds() / 3600 / 24


## Filter data to select individuals with 1 to 7 total migraine days and less than 10 attacks per month
df = (
    data_migraine.groupby(["hashed_uid", "year_month"])
    .filter(
        lambda group: (
            0 < group.shape[0] <= 10 and
            0 < group["attack_duration_days"].sum() <= 7
        )
    )
)

# Print the filtered results for debugging
print("Filtered migraine days per month (1 to 7 days):")

# Calculate the max attack_duration_days for individual episodes
print("Max attack_duration_days in data_migraine_filtered (after filtering):", df["attack_duration_days"].max())
# Group by `studyid` and calculate the total `attack_duration_days` per user in the filtered data
total_attack_duration_days_per_user = df.groupby(["hashed_uid", "year_month"])["attack_duration_days"].sum()
# Get descriptive statistics for the total migraine days per user
describe_total_attack_duration_days = total_attack_duration_days_per_user.describe()

print("Descriptive statistics for total migraine days per user (filtered):")
print(describe_total_attack_duration_days)
print("Dataset shape after filtering migraine days per month: ", df.shape)
print()

Dataset shape after filtering age:  (180916, 16)

Filtered migraine days per month (1 to 7 days):
Max attack_duration_days in data_migraine_filtered (after filtering): 7.0
Descriptive statistics for total migraine days per user (filtered):
count    51147.000000
mean         1.695387
std          1.617901
min          0.000694
25%          0.466667
50%          1.103472
75%          2.438194
max          7.000000
Name: attack_duration_days, dtype: float64
Dataset shape after filtering migraine days per month:  (119168, 20)



In [8]:
print("Dataset shape ", df.shape)

# Calculate attack duration hours for filtered dataset
df["attack_duration_hours"] = (
    df["attack_duration"].dt.total_seconds() / 3600
)

# Filter out attacks longer than 72 hours
df = df[df["attack_duration_hours"] <= 72]

# Select only migraines with durations greater than or equal to 4 hours
df = df[df["attack_duration_hours"] >= 4]

total_attack_duration_hours_per_user = df.groupby("hashed_mid")["attack_duration_hours"].sum()
# Get descriptive statistics for the total migraine days per user
describe_total_attack_duration_days = total_attack_duration_hours_per_user.describe()

print("Descriptive statistics for total migraine hours per user (filtered):")
print(describe_total_attack_duration_days)

# Print final dataset shape
print("Dataset shape after hours filtering: ", df.shape)
print()

# Drop NaN values
minimum_migraine = df.dropna()

# Print final migraine dataset summary
print("Final migraine dataset:")
print("Shape: ", minimum_migraine.shape)
print("Earliest date", minimum_migraine["creationtime_local"].dt.to_period("d").min())
print("Latest date", minimum_migraine["creationtime_local"].dt.to_period("d").max())
print("Unique users", minimum_migraine["hashed_uid"].nunique())

Dataset shape  (119168, 20)
Descriptive statistics for total migraine hours per user (filtered):
count    95092.000000
mean        17.644844
std         14.259603
min          4.000000
25%          7.416667
50%         12.583333
75%         22.916667
max         72.000000
Name: attack_duration_hours, dtype: float64
Dataset shape after hours filtering:  (95092, 21)

Final migraine dataset:
Shape:  (66815, 21)
Earliest date 2017-02-09
Latest date 2022-11-04
Unique users 3897


In [10]:
minimum_migraine

,hashed_uid,hashed_mid,age,gender,country,creationtime_local,starttime_local,endtime_local,pain_intensity,attack_type,...,reliefs,affected_activities,year,month,year_month,starttime_local_to_day,endtime_local_to_day,attack_duration,attack_duration_days,attack_duration_hours
9,ef783deb79829496809be08c0c64dc193dd45ff8bb920f...,39a4546544c1f7f9045c1b2d375105dcc6ec39f4342f18...,45.0,F,United States,2022-02-24 22:31:41.028,2022-02-22 21:36:00,2022-02-23 15:25:00,7.0,migraine,...,"hot shower,sleep",missed work,2022,2,2022-02,2022-02-22,2022-02-23,0 days 17:49:00,0.742361,17.816667
10,ef783deb79829496809be08c0c64dc193dd45ff8bb920f...,32f156710ca598df6c58e28f976c868fd85f60d1bdc712...,45.0,F,United States,2022-02-16 23:01:02.745,2022-02-11 10:01:00,2022-02-11 23:01:00,7.0,migraine,...,"dark room rest,hot bath w/ epsom,ice pack on head","hard to concentrate,missed work",2022,2,2022-02,2022-02-11,2022-02-11,0 days 13:00:00,0.541667,13.000000
15,ef783deb79829496809be08c0c64dc193dd45ff8bb920f...,0022f859c6af1eb727b03e906bee86da62fc8bafdefebc...,44.0,F,United States,2021-04-21 21:37:44.862,2021-04-21 10:38:00,2021-04-21 22:44:00,8.0,migraine,...,"dark room rest,hot bath w/ epsom,soda",missed work,2021,4,2021-04,2021-04-21,2021-04-21,0 days 12:06:00,0.504167,12.100000
17,ef783deb79829496809be08c0c64dc193dd45ff8bb920f...,2aedb20320b741d32e545df527b4b9097949cb6e6f963a...,44.0,F,United States,2021-02-26 15:32:10.764,2021-02-26 11:40:00,2021-02-26 16:09:00,3.0,migraine,...,dark room rest,hard to concentrate,2021,2,2021-02,2021-02-26,2021-02-26,0 days 04:29:00,0.186806,4.483333
21,e6b0b8e49b4b24a150e7e6e7d00715bdce7a8c27ae1731...,f42ca92db95bc8d2d95655be030a7e55471dd61b26f8f1...,44.0,F,United States,2022-03-22 06:22:47.690,2022-03-22 06:00:00,2022-03-23 13:03:00,5.0,migraine,...,"ice,sleep",missed family time,2022,3,2022-03,2022-03-22,2022-03-23,1 days 07:03:00,1.293750,31.050000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697857,a396f1b6063ec9d6bc78dd2025d4bedeccc34480b63d3a...,5b8d1ed88ea17969081b1557d085a53a417b30f3a9c700...,26.0,F,United Kingdom,2021-05-28 14:07:36.049,2021-05-28 14:07:00,2021-05-28 22:40:00,6.0,migraine,...,"drink water,food,stay indoor","hard to concentrate,missed family time,no scre...",2021,5,2021-05,2021-05-28,2021-05-28,0 days 08:33:00,0.356250,8.550000
697869,9e74f93c6b418d2c353941260667e43a213d88e099af35...,a5f3b2f4e49134cc682fd30943922aa5f0443d6249be48...,18.0,Unknown,United States,2021-06-07 02:38:59.802,2021-06-05 15:15:00,2021-06-06 00:00:00,9.0,"migraine,tension type headache",...,"dark room rest,drink water,food,ice packs,slee...","could not fall asleep,hard to concentrate,miss...",2021,6,2021-06,2021-06-05,2021-06-06,0 days 08:45:00,0.364583,8.750000
697870,d2f50b8f2397c6f4290c9df05a6a0073439d945a2a699c...,c067f65740e41b012ce0f2b7d53d8908b726d00b7f81e1...,60.0,M,United Kingdom,2021-03-09 17:35:30.278,2021-03-09 10:00:00,2021-03-09 23:59:00,2.0,migraine,...,"caffeine,dark room rest,food,sleep",not affected,2021,3,2021-03,2021-03-09,2021-03-09,0 days 13:59:00,0.582639,13.983333
697874,d2f50b8f2397c6f4290c9df05a6a0073439d945a2a699c...,aafd819d74623ca40aa80f322a2f6f8113c17fe46f4635...,60.0,M,United Kingdom,2021-01-04 09:18:49.373,2021-01-04 07:06:00,2021-01-04 14:50:00,2.0,migraine,...,"caffeine,food,sunglasses,tai chi",not affected,2021,1,2021-01,2021-01-04,2021-01-04,0 days 07:44:00,0.322222,7.733333


In [11]:
# Load sleep data
print("Loading sleep data...")
data_sleep = pd.read_csv(data_sleep_path)
data_sleep.head()

Loading sleep data...


,hashed_uid,hashed_sid,sleep_starttime_local,sleep_endtime_local,confirmed
0,faf826fb269282dc636177a488b0ca9e6275794d1d051b...,9e5729e6ad379eaa9fad8a7fea0fb44c7ef0d5e89e171d...,2019-11-01 00:57:40.863,2019-11-01 07:30:09.548,True
1,f680dd49469f512270deb2f90de4289b24d6773e3b1c24...,43c0357214f6a0f3e5828e75118bc85e9713e0ea6ceef9...,2019-11-01 04:12:00,2019-11-01 12:40:00,True
2,18d0a33df126a0781dfd5cc031e25d0315310485dd3def...,8ef415af406100af2088078d99409c07702d42bc910a90...,2019-11-01 00:37:51.213,2019-11-01 06:42:54.782,False
3,83a83c9603e8dbe4082baa8942a36b8de978139a9bc1a4...,3962000f96c1e12dea6a503f615d4a31989a1361a5ab45...,2019-11-01 00:24:51.454,2019-11-01 08:47:00,True
4,1b2d2a8ce8df25d26002dfe7731e94509109b2ef730102...,c5a6db80494cc71d5fe224f2b055ea03a08b491d558823...,2019-11-01 00:10:00,2019-11-01 07:25:00,True


In [12]:
# preprocess sleep data
# Filter users in migraine dataset
minimum_sleep = data_sleep[data_sleep["hashed_uid"].isin(minimum_migraine["hashed_uid"])]

# Convert sleep time columns to datetime and calculate sleep duration
minimum_sleep["sleep_duration"] = pd.to_datetime(minimum_sleep["sleep_endtime_local"], format="ISO8601") - pd.to_datetime(minimum_sleep["sleep_starttime_local"], format="ISO8601")
minimum_sleep["sleep_duration_hours"] = minimum_sleep["sleep_duration"].dt.total_seconds() / 3600
minimum_sleep["sleep_starttime_to_day"] = pd.to_datetime(minimum_sleep["sleep_starttime_local"], format="ISO8601").dt.to_period("D")
minimum_sleep["sleep_endtime_to_day"] = pd.to_datetime(minimum_sleep["sleep_endtime_local"], format="ISO8601").dt.to_period("D")

# Filter only confirmed cases
minimum_sleep = minimum_sleep[minimum_sleep["confirmed"] == True]

# Drop NaN values on sleep_endtime_local
minimum_sleep.dropna(subset=["sleep_endtime_local"], inplace=True)

# Select sleep duration < 24 hours
minimum_sleep = minimum_sleep[minimum_sleep["sleep_duration_hours"] < 24]

# Drop duplicates on sleep_endtime_local
minimum_sleep.drop_duplicates(subset=["hashed_uid", "sleep_endtime_local"], inplace=True)

# Drop duplicates on sleep_starttime_local
minimum_sleep.drop_duplicates(subset=["hashed_uid", "sleep_starttime_local"], inplace=True)

print("Dataset shape: ", minimum_sleep.shape)

/tmp/ipykernel_486650/3699113284.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minimum_sleep["sleep_duration"] = pd.to_datetime(minimum_sleep["sleep_endtime_local"], format="ISO8601") - pd.to_datetime(minimum_sleep["sleep_starttime_local"], format="ISO8601")
/tmp/ipykernel_486650/3699113284.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minimum_sleep["sleep_duration_hours"] = minimum_sleep["sleep_duration"].dt.total_seconds() / 3600
/tmp/ipykernel_486650/3699113284.py:8: SettingWithCopyWarning: 


Dataset shape:  (3233262, 9)


In [13]:
# Merge migraine and sleep datasets
print("Merging migraine and sleep datasets...")
merge_minimum_migraine_sleep = pd.merge(
    minimum_migraine,
    minimum_sleep,
    left_on=["hashed_uid", "starttime_local_to_day"],
    right_on=["hashed_uid", "sleep_endtime_to_day"],
    how="inner",
)

# Drop duplicates after merging
merge_minimum_migraine_sleep.drop_duplicates(subset=["hashed_uid", "starttime_local_to_day"], inplace=True)
merge_minimum_migraine_sleep.drop_duplicates(subset=["hashed_uid", "endtime_local_to_day"], inplace=True)

# Print merged dataset summary
print("Final merged dataset:")
print("Shape: ", merge_minimum_migraine_sleep.shape)
print("Earliest date", merge_minimum_migraine_sleep["creationtime_local"].dt.to_period("d").min())
print("Latest date", merge_minimum_migraine_sleep["creationtime_local"].dt.to_period("d").max())
print("Unique users", merge_minimum_migraine_sleep["hashed_uid"].nunique())

# Save the final merged dataset to a CSV file
merge_minimum_migraine_sleep.to_csv(final_output_path, index=False)

Merging migraine and sleep datasets...
Final merged dataset:
Shape:  (57035, 29)
Earliest date 2018-07-01
Latest date 2022-11-04
Unique users 3652
